In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_parquet('options_data/OptionMetrics.parquet', engine='pyarrow')  # or engine='fastparquet'
print(df)


          secid        date             symbol  symbol_flag      exdate  \
0        101121  2022-08-31  AMD 220902C100000            1  2022-09-02   
1        101121  2022-08-31  AMD 220902C101000            1  2022-09-02   
2        101121  2022-08-31  AMD 220902C102000            1  2022-09-02   
3        101121  2022-08-31  AMD 220902C103000            1  2022-09-02   
4        101121  2022-08-31  AMD 220902C104000            1  2022-09-02   
...         ...         ...                ...          ...         ...   
6541603  213572  2023-08-31    ZM 230929P79000            1  2023-09-29   
6541604  213572  2023-08-31    ZM 230929P80000            1  2023-09-29   
6541605  213572  2023-08-31    ZM 230929P85000            1  2023-09-29   
6541606  213572  2023-08-31    ZM 230929P90000            1  2023-09-29   
6541607  213572  2023-08-31    ZM 230929P95000            1  2023-09-29   

          last_date cp_flag  strike_price  best_bid  best_offer  ...     sic  \
0        2022-08-31

In [3]:
df.sample(10).head(10)

,secid,date,symbol,symbol_flag,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,...,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer,div_convention,exercise_style,am_set_flag
3715818,107899,2023-02-28,QQQ 230314P240000,1,2023-03-14,2023-02-28,P,240000,0.03,0.04,...,NaN,0,4,None,%,426,INVESCO CAPITAL MANAGEMENT L,None,A,None
4427957,109820,2023-01-04,SPY 230120C720000,1,2023-01-20,2022-12-30,C,720000,0.00,0.01,...,NaN,0,1,None,%,None,SSGA ACTIVE TRUST,None,A,None
197498,102936,2022-10-12,JPM 221104C106000,1,2022-11-04,2022-10-12,C,106000,3.65,3.85,...,6021.0,0,1,None,0,None,JPMORGAN CHASE & CO.,None,A,None
2960964,104533,2023-05-09,XOM 230602P117000,1,2023-06-02,2023-05-04,P,117000,8.80,8.95,...,2911.0,0,1,None,0,None,EXXON MOBIL CORP.,None,A,None
6538578,213572,2023-08-21,ZM 230915C66000,1,2023-09-15,2023-08-21,C,66000,5.05,5.15,...,7370.0,0,4,None,0,None,ZOOM VIDEO COMMUNICATIONS IN,None,A,None
2013222,121812,2022-12-08,GOOGL 221230C98000,1,2022-12-30,2022-12-08,C,98000,1.42,1.45,...,7370.0,0,4,None,0,None,ALPHABET INC,None,A,None
1838083,113993,2022-11-08,GME 221118C24500,1,2022-11-18,2022-11-08,C,24500,1.73,1.89,...,5734.0,0,1,None,0,None,GAMESTOP CORPORATION,None,A,None
615908,106566,2022-12-22,JNJ 221223C90000,1,2022-12-23,None,C,90000,86.65,87.75,...,2834.0,0,1,None,0,None,JOHNSON & JOHNSON,None,A,None
3844666,107899,2023-05-15,QQQ 230602P299000,1,2023-06-02,2023-05-15,P,299000,0.37,0.39,...,NaN,0,4,None,%,426,INVESCO CAPITAL MANAGEMENT L,None,A,None
1381015,109820,2022-12-22,SPY 230104C385000,1,2023-01-04,2022-12-22,C,385000,3.05,3.08,...,NaN,0,1,None,%,None,SSGA ACTIVE TRUST,None,A,None


In [4]:
# split symbol after space
df["ticker"] = df["symbol"].str.split(" ").str[0]

In [5]:
unq_tickers = df["ticker"].unique()

In [6]:
tikcers_of_interest = [
    "JPM",
    "BAC",
    "WFC",
    "PFE",
    "MRK",
    "HD",
    "MCD",
    "NKE",
    "XOM",
    "CVX",
    "COP",
    "AMD",
    "ZM",
    "GME",
    "AMC",
    "BB",
    "NOK",]

In [9]:
df = df.loc[df["ticker"].isin(tikcers_of_interest) & (df["cp_flag"] == "C")]

In [11]:
def get_prices(tickers):
    files = os.listdir("options_data")
    ticker_data = pd.DataFrame()
    for file in files:
        if any(file.endswith(f"{t}.csv") for t in tickers):
            ticker = file.split(".csv")[0]
            sub_df = pd.read_csv(f"options_data/{file}", index_col=0, parse_dates=True)
            sub_df.rename(columns={"Close": f"{ticker}"}, inplace=True)
            ticker_data = ticker_data.join(sub_df, how='right')
    prices = ticker_data
    volatilities = ticker_data.pct_change().rolling(window=100).std()
    volatilities = volatilities.dropna()
    prices = prices.loc[volatilities.index]
    return prices, volatilities

prices, _ = get_prices(tikcers_of_interest)

In [14]:
def assign_s(row):
    ticker = row["ticker"]
    date = row["date"]
    return prices.loc[date, ticker]

df["S"] = df.loc[df["date"] == "2023-05-04"].apply(assign_s, axis=1)

/tmp/ipykernel_1504/3250371332.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["S"] = df.loc[df["date"] == "2023-05-04"].apply(assign_s, axis=1)


In [17]:
day_df = df.loc[df["date"] == "2023-05-04"]
day_df["S_prime"] = day_df["S"] / (day_df["strike_price"]/100)

/tmp/ipykernel_1504/1300182572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_df["S_prime"] = day_df["S"] / (day_df["strike_price"]/100)


In [24]:
day_df["ticker"].loc[day_df["best_bid"] - (day_df["S"] - (day_df["strike_price"]/100)) < 0].unique()

array(['AMC'], dtype=object)

In [25]:
day_df["ticker"].unique()

array(['AMD', 'BAC', 'JPM', 'CVX', 'XOM', 'HD', 'MCD', 'MRK', 'NKE',
       'NOK', 'PFE', 'COP', 'BB', 'WFC', 'GME', 'AMC', 'ZM'], dtype=object)